In [19]:
train_path = './imagesets/A/train.txt'
test_path = './imagesets/A/test.txt'
val_path = './imagesets/A/val.txt'

save_path = '/home/newDisk/tool/carla_dataset_tool/dataset/'

# read imageset(train_path, test_path, val_path)
train_set = set()
test_set = set()
val_set = set()

with open(train_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        train_set.add(line.replace('\n',''))

with open(test_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        test_set.add(line.replace('\n',''))

with open(val_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
        val_set.add(line.replace('\n',''))

set_A_05 = ['0121_0157', '0121_0204', '0121_0211', '0121_0219', '0121_0227']
set_B_02 = ['0104_2223', '0104_2309', '0104_2328', '0104_2343', '0105_0013']
set_C_10 = ['0116_0117', '0116_0125', '0116_0132', '0116_0140', '0116_0149']
set_D_06 = ['0104_1949', '0104_2002', '0104_2016', '0104_2032', '0104_2056']

current_set = set_A_05  

In [20]:
import os
import shutil
import argparse

proj_path = '/home/newDisk/tool/carla_dataset_tool/'
dir_set = []

for i in range(len(current_set)):
    subset = current_set[i]
    dir = '/home/newDisk/tool/carla_dataset_tool/raw_data/record_2024_'+subset
    dir_set.append(dir)


# get inner frame id, look if both exist label and lidar file
lidar_ckeck = [path+'/vehicle.tesla.model3.master/velodyne/' for path in dir_set]
label_check = [path+'/vehicle.tesla.model3.master/velodyne_semantic/' for path in dir_set]
image_check = [path+'/vehicle.tesla.model3.master/image_2/' for path in dir_set]
calib_path = proj_path + 'test_data/000001.txt'


lidar_target_path = {'train': proj_path+'dataset/training/velodyne/', 'test': proj_path+'dataset/testing/velodyne/'}
image_target_path = {'train': proj_path+'dataset/training/image_2/', 'test': proj_path+'dataset/testing/image_2/'}
label_target_path = {'train': proj_path+'dataset/training/label_2/', 'test': proj_path+'dataset/testing/label_2/'}
calib_target_path = {'train': proj_path+'dataset/training/calib/', 'test': proj_path+'dataset/testing/calib/'}

# create folder if not exist
for key in lidar_target_path.keys():
    if not os.path.exists(lidar_target_path[key]):
        os.makedirs(lidar_target_path[key])
    if not os.path.exists(image_target_path[key]):
        os.makedirs(image_target_path[key])
    if not os.path.exists(label_target_path[key]):
        os.makedirs(label_target_path[key])
    if not os.path.exists(calib_target_path[key]):
        os.makedirs(calib_target_path[key])



for train_frame in train_set:
    lidar_name = '00000' + train_frame[-5:]+'.bin'
    label_name = '00000' + train_frame[-5:]+'.txt'
    image_name = '00000' + train_frame[-5:]+'.png'

    lidar_dir = lidar_ckeck[int(train_frame[0])]
    label_dir = label_check[int(train_frame[0])]
    image_dir = image_check[int(train_frame[0])]
    # check if file exist
    if os.path.exists(image_dir+image_name):
        image_file_path = os.path.join(image_dir, image_name)
    else:
        image_file_path = '/home/newDisk/tool/carla_dataset_tool/raw_data/record_2024_0104_1949/vehicle.tesla.model3.master/image_2/0000012836.png'
    if os.path.exists(lidar_dir+lidar_name) and os.path.exists(label_dir+label_name):
        # copy lidar file
        shutil.copy(os.path.join(lidar_dir, lidar_name), os.path.join(lidar_target_path['train'], train_frame+'.bin'))
        
        # copy label file
        with open(os.path.join(label_dir, label_name), 'r') as f:
            lines = f.readlines()
            lines = lines[:-1]
            fix_lines = ""
            for line in lines:
                elements = line.split(" ")
                # print('This is the test version, when fix the recorder, update here')
                # x, y, z, l, w, h, rot, lab, _, _, _ = elements
                x, y, z, l, w, h, rot, lab, _, _ = elements
                l = abs(round(float(l),4))
                w = abs(round(float(w),4))
                h = abs(round(float(h),4))
                x = round(float(x),4)
                y = round(float(y),4)
                z = round(float(z),4)
                rot = round(float(rot),4)
                # if lab == 'Bus': lab = 'Van'
                # x, y, z, dx, dy, dz, rot, lab, _, _, _ = elements
                rot -= 3.14 if rot > 3.14 else 0
                rot = round(float(rot),4)
                fix_line = "{} {} {} {} {} {} {} {} {} {} {} {} {} {} {}\n".format(lab, '0', '0', '0', '0', '0', '0', '0', h, w, l, x, y, z, rot)
                # fix_line = "{} {} {} {} {} {} {} {}\n".format(x,y,z,dx,dy,dz,rot,lab)
                fix_lines += fix_line
        with open(os.path.join(label_target_path['train'], train_frame+'.txt'), 'w') as f:
            f.writelines(fix_lines)
        
        # copy image file
        shutil.copy(image_file_path, os.path.join(image_target_path['train'], train_frame+'.png'))
    
        # copy calib file
        shutil.copy(calib_path, os.path.join(calib_target_path['train'], train_frame+'.txt'))
    
    else:
        print('file not exist')
        print(train_frame)


In [21]:
for train_frame in test_set:
    lidar_name = '00000' + train_frame[-5:]+'.bin'
    label_name = '00000' + train_frame[-5:]+'.txt'
    image_name = '00000' + train_frame[-5:]+'.png'
    lidar_dir = lidar_ckeck[int(train_frame[0])]
    label_dir = label_check[int(train_frame[0])]
    image_dir = image_check[int(train_frame[0])]
    # check if file exist
    if os.path.exists(image_dir+image_name):
        image_file_path = os.path.join(image_dir, image_name)
    else:
        image_file_path = '/home/newDisk/tool/carla_dataset_tool/raw_data/record_2024_0104_1949/vehicle.tesla.model3.master/image_2/0000012836.png'
    if os.path.exists(lidar_dir+lidar_name) and os.path.exists(label_dir+label_name):
        # copy lidar file
        shutil.copy(os.path.join(lidar_dir, lidar_name), os.path.join(lidar_target_path['test'], train_frame+'.bin'))
        
        # copy label file
        with open(os.path.join(label_dir, label_name), 'r') as f:
            lines = f.readlines()
            lines = lines[:-1]
            fix_lines = ""
            for line in lines:
                elements = line.split(" ")
                # print('This is the test version, when fix the recorder, update here')
                # x, y, z, l, w, h, rot, lab, _, _, _ = elements
                x, y, z, l, w, h, rot, lab, _, _ = elements
                l = abs(round(float(l),4))
                w = abs(round(float(w),4))
                h = abs(round(float(h),4))
                x = round(float(x),4)
                y = round(float(y),4)
                z = round(float(z),4)
                rot = round(float(rot),4)
                # if lab == 'Bus': lab = 'Van'
                # x, y, z, dx, dy, dz, rot, lab, _, _, _ = elements
                rot -= 3.14 if rot > 3.14 else 0
                rot = round(float(rot),4)
                fix_line = "{} {} {} {} {} {} {} {} {} {} {} {} {} {} {}\n".format(lab, '0', '0', '0', '0', '0', '0', '0', h, w, l, x, y, z, rot)
                # fix_line = "{} {} {} {} {} {} {} {}\n".format(x,y,z,dx,dy,dz,rot,lab)
                fix_lines += fix_line
        with open(os.path.join(label_target_path['test'], train_frame+'.txt'), 'w') as f:
            f.writelines(fix_lines)
        
        # copy image file
        shutil.copy(image_file_path, os.path.join(image_target_path['test'], train_frame+'.png'))
    
        # copy calib file
        shutil.copy(calib_path, os.path.join(calib_target_path['test'], train_frame+'.txt'))
    
    else:
        print('file not exist')
        print(train_frame)

In [22]:
for train_frame in val_set:
    lidar_name = '00000' + train_frame[-5:]+'.bin'
    label_name = '00000' + train_frame[-5:]+'.txt'
    image_name = '00000' + train_frame[-5:]+'.png'
    lidar_dir = lidar_ckeck[int(train_frame[0])]
    label_dir = label_check[int(train_frame[0])]
    image_dir = image_check[int(train_frame[0])]
    # check if file exist
    if os.path.exists(image_dir+image_name):
        image_file_path = os.path.join(image_dir, image_name)
    else:
        image_file_path = '/home/newDisk/tool/carla_dataset_tool/raw_data/record_2024_0104_1949/vehicle.tesla.model3.master/image_2/0000012836.png'
    if os.path.exists(lidar_dir+lidar_name) and os.path.exists(label_dir+label_name):
        # copy lidar file
        shutil.copy(os.path.join(lidar_dir, lidar_name), os.path.join(lidar_target_path['train'], train_frame+'.bin'))
        
        # copy label file
        with open(os.path.join(label_dir, label_name), 'r') as f:
            lines = f.readlines()
            lines = lines[:-1]
            fix_lines = ""
            for line in lines:
                elements = line.split(" ")
                # print('This is the test version, when fix the recorder, update here')
                # x, y, z, l, w, h, rot, lab, _, _, _ = elements
                x, y, z, l, w, h, rot, lab, _, _ = elements
                l = abs(round(float(l),4))
                w = abs(round(float(w),4))
                h = abs(round(float(h),4))
                x = round(float(x),4)
                y = round(float(y),4)
                z = round(float(z),4)
                rot = round(float(rot),4)
                # if lab == 'Bus': lab = 'Van'
                # x, y, z, dx, dy, dz, rot, lab, _, _, _ = elements
                rot -= 3.14 if rot > 3.14 else 0
                rot = round(float(rot),4)
                fix_line = "{} {} {} {} {} {} {} {} {} {} {} {} {} {} {}\n".format(lab, '0', '0', '0', '0', '0', '0', '0', h, w, l, x, y, z, rot)
                # fix_line = "{} {} {} {} {} {} {} {}\n".format(x,y,z,dx,dy,dz,rot,lab)
                fix_lines += fix_line
        with open(os.path.join(label_target_path['train'], train_frame+'.txt'), 'w') as f:
            f.writelines(fix_lines)
        
        # copy image file
        shutil.copy(image_file_path, os.path.join(image_target_path['train'], train_frame+'.png'))
    
        # copy calib file
        shutil.copy(calib_path, os.path.join(calib_target_path['train'], train_frame+'.txt'))
    
    else:
        print('file not exist')
        print(train_frame)

In [23]:
# copy imageset

image_set_path = proj_path + 'dataset/ImageSets/'
if not os.path.exists(image_set_path):
    os.makedirs(image_set_path)

shutil.copy(train_path, image_set_path+'train.txt')
shutil.copy(test_path, image_set_path+'test.txt')
shutil.copy(val_path, image_set_path+'val.txt')

'/home/newDisk/tool/carla_dataset_tool/dataset/ImageSets/val.txt'

In [24]:
# deprecated

In [25]:

# # copy lidar
# target_path = {'train': proj_path+'dataset/training/velodyne/', 'test': proj_path+'dataset/testing/velodyne/'}
# for key in target_path:
#     if not os.path.exists(target_path[key]):
#         os.makedirs(target_path[key])
# source_path = []
# for i in dir_set:
#     source_path.append(i+'/vehicle.tesla.model3.master/velodyne/')

# for train_frame in train_set:
#     lidar_name = '00000' + train_frame[-5:]+'.bin'
#     dir = source_path[int(train_frame[0])]
#     file_path = dir+lidar_name
#     shutil.copy(os.path.join(dir, lidar_name), os.path.join(target_path['train'], train_frame+'.bin'))

# for test_frame in test_set:
#     lidar_name = '00000' + test_frame[-5:]+'.bin'
#     dir = source_path[int(test_frame[0])]
#     file_path = dir+lidar_name
#     shutil.copy(os.path.join(dir, lidar_name), os.path.join(target_path['test'], test_frame+'.bin'))

# for val_frame in val_set:
#     lidar_name = '00000' + val_frame[-5:]+'.bin'
#     dir = source_path[int(val_frame[0])]
#     file_path = dir+lidar_name
#     shutil.copy(os.path.join(dir, lidar_name), os.path.join(target_path['train'], val_frame+'.bin'))

In [26]:
# # copy image
# target_path = {'train': proj_path+'dataset/training/image_2/', 'test': proj_path+'dataset/testing/image_2/'}
# for key in target_path:
#     if not os.path.exists(target_path[key]):
#         os.makedirs(target_path[key])
# source_path = []
# for i in dir_set:
#     source_path.append(i+'/vehicle.tesla.model3.master/image_2/')
# for train_frame in train_set:
#     lidar_name = '00000' + train_frame[-5:]+'.png'
#     dir = source_path[int(train_frame[0])]
#     file_path = dir+lidar_name
#     shutil.copy(os.path.join(dir, lidar_name), os.path.join(target_path['train'], train_frame+'.png'))
# for test_frame in test_set:
#     lidar_name = '00000' + test_frame[-5:]+'.png'
#     dir = source_path[int(test_frame[0])]
#     file_path = dir+lidar_name
#     shutil.copy(os.path.join(dir, lidar_name), os.path.join(target_path['test'], test_frame+'.png'))
# for val_frame in val_set:
#     lidar_name = '00000' + val_frame[-5:]+'.png'
#     dir = source_path[int(val_frame[0])]
#     file_path = dir+lidar_name
#     shutil.copy(os.path.join(dir, lidar_name), os.path.join(target_path['train'], val_frame+'.png'))
    